In [17]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [18]:
torch.manual_seed(1337)

In [1]:
#read the data
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

print('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


In [2]:
#first 1000 characters
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [3]:
#unique characters in the file
chars = sorted(list(set(text)))
print('{} unique characters'.format(len(chars)))
vocab_size = len(chars)
print('-'.join(chars))
print('vocab_size = {}'.format(vocab_size))

65 unique characters

- -!-$-&-'-,---.-3-:-;-?-A-B-C-D-E-F-G-H-I-J-K-L-M-N-O-P-Q-R-S-T-U-V-W-X-Y-Z-a-b-c-d-e-f-g-h-i-j-k-l-m-n-o-p-q-r-s-t-u-v-w-x-y-z
vocab_size = 65


In [6]:
#create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode('hello'))
print(decode(encode('hello')))

#there can be many ways to tokenize the text based on merits and demerits (trade-offs)

[46, 43, 50, 50, 53]
hello


In [8]:
# Let's now encode the entire text dataset and store it as a torch tensor
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [9]:
#split the data into train and test
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [11]:
block_size = 8 # spatial extent of the model for its context window (how many characters to look back) 
train_data[:block_size+1] # first 9 characters (first 8 plus the 9th for the target) 

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [14]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f'When input is {context}, target is {[target]}')

When input is tensor([18]), target is [tensor(47)]
When input is tensor([18, 47]), target is [tensor(56)]
When input is tensor([18, 47, 56]), target is [tensor(57)]
When input is tensor([18, 47, 56, 57]), target is [tensor(58)]
When input is tensor([18, 47, 56, 57, 58]), target is [tensor(1)]
When input is tensor([18, 47, 56, 57, 58,  1]), target is [tensor(15)]
When input is tensor([18, 47, 56, 57, 58,  1, 15]), target is [tensor(47)]
When input is tensor([18, 47, 56, 57, 58,  1, 15, 47]), target is [tensor(58)]


In [16]:
batch_size = 4 # number of sequences in a mini-batch (number of samples processed in parallel)
block_size = 8  # length of a sequence (number of tokens in a sample) 

def get_batch(split):
    #genertae a small batch of data of input x and target y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print(xb.shape, yb.shape)
print(xb)
print(yb)

print('-----------------')

for b in range(batch_size):
    print(f'Batch {b+1}')
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f'When input is {context}, target is {[target]}')

torch.Size([4, 8]) torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
-----------------
Batch 1
When input is tensor([24]), target is [tensor(43)]
When input is tensor([24, 43]), target is [tensor(58)]
When input is tensor([24, 43, 58]), target is [tensor(5)]
When input is tensor([24, 43, 58,  5]), target is [tensor(57)]
When input is tensor([24, 43, 58,  5, 57]), target is [tensor(1)]
When input is tensor([24, 43, 58,  5, 57,  1]), target is [tensor(46)]
When input is tensor([24, 43, 58,  5, 57,  1, 46]), target is [tensor(43)]
When input is tensor([24, 43, 58,  5, 57,  1, 46, 43]), target is [tensor(39)]
Batch 2
When input is tensor([44]), target is [tensor(53)]
When input is tensor(

In [30]:
class BigramLanguageModel(nn.Module):
    
    def __init__(self,vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    
    def forward(self, idx, targets=None):

        #idx and targets are both of shape (batch_size, block_size)
        logits = self.token_embedding_table(idx) # shape (batch_size, block_size, vocab_size)
        #B, T, C = logits.shape
        #logits = logits.view(B*T, C) # shape (batch_size*block_size, vocab_size)
        #targets = targets.view(-1) # shape (batch_size*block_size,)
        if targets is not None:
            loss = F.cross_entropy(logits.view(-1, vocab_size), targets.view(-1))
        else:
            loss = None
        
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        #idx is of shape (batch_size, block_size) and contains the starting tokens for each sequence
        for _ in range(max_new_tokens):
            #get the logits for the next token
            logits, _ = self(idx)
            # focus on the last token for each sequence
            logits = logits[:, -1, :] # shape (batch_size, vocab_size)
            # sample the next token from the probability distribution for each sequence in the batch 
            # (we use multinomial instead of argmax here to introduce some randomness and avoid loops)
            probs = F.softmax(logits, dim=-1) # shape (batch_size, vocab_size)
            idx_next = torch.multinomial(probs, num_samples=1) # shape (batch_size, 1)
            idx = torch.cat([idx, idx_next], dim=1) # shape (batch_size, block_size+1)
        return idx
    
m = BigramLanguageModel(vocab_size)
out, loss = m(xb, yb)
print(out.shape)
print(loss)
print(decode(m.generate((torch.zeros((1,1), dtype=torch.long)), max_new_tokens=100)[0].tolist()))

torch.Size([4, 8, 65])
tensor(4.3173, grad_fn=<NllLossBackward0>)

xnmfn'Qm:F$D! 
lS:EsG;$
ralL,!ODnzi;rdYC?Xm3pRzV'RsH hq:OLAtbCUMXBMd'px,M'xhoi
GXNUSFyv'ulEb,bCS:XCX


In [31]:
optimizer = torch.optim.Adam(m.parameters(), lr=1e-3)

In [36]:
batch_size = 32
for steps in range(10000):
    #get a batch of data
    x, y = get_batch('train')
    #forward pass
    logits, loss = m(x, y)
    #backward pass
    loss.backward()
    #update parameters
    optimizer.step()
    optimizer.zero_grad()
    if steps % 1000 == 0:
        print(f'Step {steps}, loss = {loss.item()}')

Step 0, loss = 2.4978713989257812
Step 1000, loss = 2.388235330581665
Step 2000, loss = 2.4288995265960693
Step 3000, loss = 2.492995500564575
Step 4000, loss = 2.6295342445373535
Step 5000, loss = 2.421825647354126
Step 6000, loss = 2.4721992015838623
Step 7000, loss = 2.5137646198272705
Step 8000, loss = 2.445064067840576
Step 9000, loss = 2.4209587574005127


In [38]:
print(decode(m.generate((torch.zeros((1,1), dtype=torch.long)), max_new_tokens=200)[0].tolist()))


Le IIO: iurqund My,
Thf r n h!
QUS:
NGo
Thth


THAUCatowie werdesh, th,--f s mitot ust ans? Y:
VIs, myo be s p d, k pilll geinget wh
Thingarat
MO,
He the; omescor wst oro bois inde; RK:
IV:
ABO Ris, o


In [40]:
# Mathematical trick in self attention

B, T, C = 4, 8, 2 # batch size, sequence length, channels
x = torch.randn(B, T, C) # random input
x.shape

torch.Size([4, 8, 2])

In [41]:
# We want x[b,t] = mean_{i<=t} x[b,i] for all b,t
xbow = torch.zeros(B, T, C) # output
for b in range(B):
    for t in range(T):
        xbow[b,t] = x[b,:t+1].mean(dim=0)

In [42]:
x[0]

tensor([[-0.4576,  0.9253],
        [-0.0093,  1.2836],
        [-0.8215,  0.0609],
        [-0.6706, -1.4099],
        [ 0.4213, -0.8366],
        [ 0.7078, -2.0029],
        [-1.2218, -0.6679],
        [-0.1762, -0.9599]])

In [43]:
xbow[0]

tensor([[-0.4576,  0.9253],
        [-0.2335,  1.1045],
        [-0.4295,  0.7566],
        [-0.4898,  0.2150],
        [-0.3075,  0.0047],
        [-0.1383, -0.3299],
        [-0.2931, -0.3782],
        [-0.2785, -0.4509]])

In [46]:
a = torch.ones(3,3)
b = torch.randint(0, 10, (3,2)).float()
c= a @ b
print('a = ', a)
print('-----------------')
print('b = ', b)
print('-----------------')
print('c = ', c)


a =  tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])
-----------------
b =  tensor([[8., 0.],
        [1., 3.],
        [0., 3.]])
-----------------
c =  tensor([[9., 6.],
        [9., 6.],
        [9., 6.]])


In [49]:
a = torch.tril(torch.ones(3,3))
a = a/a.sum(dim=1, keepdim=True)
b = torch.randint(0, 10, (3,2)).float()
c= a @ b
print('a = ', a)
print('-----------------')
print('b = ', b)
print('-----------------')
print('c = ', c)

a =  tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
-----------------
b =  tensor([[4., 5.],
        [5., 7.],
        [7., 0.]])
-----------------
c =  tensor([[4.0000, 5.0000],
        [4.5000, 6.0000],
        [5.3333, 4.0000]])


In [50]:
#v2
wei = torch.tril(torch.ones(T,T))
wei = wei/wei.sum(dim=1, keepdim=True)
xbow2 = wei @ x # shape (B, T, C) 
torch.allclose(xbow, xbow2)

True

In [51]:
#v3
tril = torch.tril(torch.ones(T,T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril==0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x # shape (B, T, C)

torch.allclose(xbow, xbow3)

True